MODEL TRAINING

In [43]:
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

Leemos el dataset limpio

In [18]:
df = pd.read_csv('../data/MLA_100k_clean.csv')


In [19]:
df.head()

,condition,base_price,shipping,non_mercado_pago_payment_methods,price,buying_mode,descriptions,last_updated,accepts_mercadopago,date_created,stop_time,status,initial_quantity,start_time,sold_quantity,available_quantity
0,new,80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",80.0,buy_it_now,[{'id': 'MLA578052519-912855983'}],20150905,True,20150905,20151104,active,1,20150905,0,1
1,used,2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",2650.0,buy_it_now,[{'id': 'MLA581565358-930764806'}],20150926,True,20150926,20151125,active,1,20150926,0,1
2,used,60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",60.0,buy_it_now,[{'id': 'MLA578780872-916478256'}],20150909,True,20150909,20151108,active,1,20150909,0,1
3,new,580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",580.0,buy_it_now,[{'id': 'MLA581877385-932309698'}],20151005,True,20150928,20151204,active,1,20150928,0,1
4,used,30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",30.0,buy_it_now,[{'id': 'MLA576112692-902981678'}],20150828,True,20150824,20151023,active,1,20150824,0,1


generar los modelos de predicción utilizando diferentes algoritmos: Regresión Logística, RandomForestClassifier, y XGBClassifier. 

In [30]:
# Definir X e y
X = df.drop('condition', axis=1)  # Ajusta el target a 'condition'
y = df['condition']

# Eliminar columnas no numéricas o complejas
X = X.drop(columns=['shipping', 'non_mercado_pago_payment_methods', 'descriptions'], errors='ignore')

# Aplicar One-Hot Encoding a las columnas categóricas
X = pd.get_dummies(X)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Convertir las clases categóricas 'new' y 'used' a valores numéricos
y = y.map({'new': 1, 'used': 0})

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train, y_train)

# Hacer predicciones con XGBoost
y_pred_xgb = xgb_clf.predict(X_test)

# Evaluar el modelo
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'Accuracy XGBoost: {accuracy_xgb}')


c:\Users\MAFE\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:10:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy XGBoost: 0.78175


#### Entrenamos los Modelos

1. Regresión Logística


In [33]:
# Verifica si hay columnas con datos categóricos
print(X.dtypes)

# Si hay columnas categóricas, puedes usar One-Hot Encoding
X = pd.get_dummies(X)


base_price                float64
price                     float64
last_updated                int64
accepts_mercadopago          bool
date_created                int64
stop_time                   int64
initial_quantity            int64
start_time                  int64
sold_quantity               int64
available_quantity          int64
buying_mode_auction          bool
buying_mode_buy_it_now       bool
buying_mode_classified       bool
status_active                bool
status_closed                bool
status_not_yet_active        bool
status_paused                bool
dtype: object


In [38]:
# Crear el modelo de Regresión Logística
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Hacer predicciones con Regresión Logística
y_pred_log = log_reg.predict(X_test)
accuracy_log = accuracy_score(y_test, y_pred_log)
print(f'Accuracy Logistic Regression: {accuracy_log}')

Accuracy Logistic Regression: 0.7095


RandomForestClassifier

In [39]:
# Crear el modelo Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_clf.fit(X_train, y_train)

# Hacer predicciones
y_pred_rf = rf_clf.predict(X_test)

# Evaluar el modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy Random Forest: {accuracy_rf}')


Accuracy Random Forest: 0.76875


XGBClassifier

In [40]:
# Crear el modelo XGBoost
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Entrenar el modelo
xgb_clf.fit(X_train, y_train)

# Hacer predicciones
y_pred_xgb = xgb_clf.predict(X_test)

# Evaluar el modelo
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'Accuracy XGBoost: {accuracy_xgb}')


c:\Users\MAFE\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:14:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy XGBoost: 0.78175


Comparar los resultados

In [41]:
print(f'Accuracy Logistic Regression: {accuracy_log}')
print(f'Accuracy Random Forest: {accuracy_rf}')
print(f'Accuracy XGBoost: {accuracy_xgb}')


Accuracy Logistic Regression: 0.7095
Accuracy Random Forest: 0.76875
Accuracy XGBoost: 0.78175


otras metricas de evaluación

In [42]:
from sklearn.metrics import classification_report

# Evaluación para Regresión Logística
print("Logistic Regression Report")
print(classification_report(y_test, y_pred_log))

# Evaluación para Random Forest
print("Random Forest Report")
print(classification_report(y_test, y_pred_rf))

# Evaluación para XGBoost
print("XGBoost Report")
print(classification_report(y_test, y_pred_xgb))


Logistic Regression Report
              precision    recall  f1-score   support

           0       0.62      0.95      0.75      9283
           1       0.92      0.50      0.65     10717

    accuracy                           0.71     20000
   macro avg       0.77      0.73      0.70     20000
weighted avg       0.78      0.71      0.70     20000

Random Forest Report
              precision    recall  f1-score   support

           0       0.72      0.81      0.76      9283
           1       0.82      0.73      0.77     10717

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000

XGBoost Report
              precision    recall  f1-score   support

           0       0.73      0.85      0.78      9283
           1       0.85      0.72      0.78     10717

    accuracy                           0.78     20000
   macro avg       0.79      0.79      0.78     20000
weighted av

Despues de entrenar los tres modelos: Regresión Logística, RandomForestClassifier, y XGBClassifier. Para seleccionar el mejor, usamos la métrica de accuracy.

En este caso el modelo de XGBoost ha tenido el mejor rendimiento en términos de accuracy y otras métricas. (0.78175)

### Guardar el modelo XGBoost como archivo .pkl

In [45]:
joblib.dump(xgb_clf, '../Model/modelo_xgboost.pkl')

print("Modelo XGBoost guardado como 'modelo_xgboost.pkl'")

Modelo XGBoost guardado como 'modelo_xgboost.pkl'
